# Mu2E Field Mapping Project

## Quick Mu2E General Overview

The primary goal of the Mu2E experiment at Fermilab is to detect or place limits on neutrino-less muon to electron conversions.  Muons will be created from proton collisions at one end of the experiment, then will travel through a series of electromagnets and into an aluminum target.  They will form a temporary bound state with the aluminum atoms, and then decay to an electron (and two neutrinos).  The resultant electron will spiral through a tracker in a strong magnetic field, such that its energy and momentum will be measured with high precision.  If no neutrinos are produced during the muon decay, the electron will inherit almost all of the muon's mass-energy, which will look slightly different than the spectrum of electron energies from regular muon decay.  Compared to previous experiments dedicated to this search, Mu2E will be ~10000x more sensitive.  In order to reach such a high sensitivity, the magnetic fields of the experiment must be known to an extreme degree of accuracy and precision.

The goal of the Field Mapping System (FMS) is to measure the magnetic field inside of the Production and Detection Solenoid (PS and DS), and then reconstruct a continuous magnetic field from those measurements.  The main errors in that magnetic field reconstruction should stem from the physical limitations of the measurement devices, and **not** from the numeric methods that use the measurements.

The magnetic field reconstruction is done through use of a semi-analytical vector field function, based on the solution to Laplace's equation.  The function is a series expansion in terms of trig functions and bessel functions, where the coefficients are free parameters that are determined by fitting this function to the measured points.

Below is a proposed Mu2E Schematic from ~2012 (basic design is still valid):

![Mu2E Schematic](Mu2e-Solenoid-Main-Assembly-2012-01-31.png)

The DS starts at a Z of 3239 and extends to 13989.  Muons are incident from the left, after traveling through the two quarter toroids that make up the Transition Solenoid (TS).  They strike the aluminum target block, located at ~6000-6300, where they are captured by aluminum atoms and maintain a stable orbit.  Once they decay, the resultant electron spirals through the constant region of the detector (from ~8000 to ~13000), leaving hits in the tracker annuli located from ~8400 to ~11660.  The tracker region must be known the most precisely, as this is the region that will be used to reconstruct the electrons.

In order to quantify the effects of the magnetic field reconstruction, we need to start examining the simulated fields.  We will pull up the Mau10 field simulation for the DS and start examining the distributions.

## Examining the Field Distributions

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
from mu2e.datafileprod import DataFileMaker
from mu2e.plotter import *

In [ ]:
init_notebook_mode()

In [ ]:
df = DataFileMaker('../datafiles/Mau10/Standard_Maps/Mu2e_DSMap',use_pickle = True).data_frame

We'll be using a pandas DataFrame object to store the magnetic field information for the DS.  The Mau10 map has been processed such that cylindrical coordinates and a cylindrical vector field were generated from the initial Cartesian information.  We've also recentered the X coordinate, such that there is an equal number of steps above and below the X=0 line.  This may **not** be equivalent to recentering the DS on the geometric center, however (we will revisit this issue later, and the complications that arise because of it).

In [ ]:
df.head()

Obviously, a table of numbers like the one above is unwieldy and does not give us a good insight on the nature of the magnetic field.  Let's make some 3D plots instead.

In [ ]:
plot_maker = Plotter({'DS_Mau10':df})

In [ ]:
plot_maker.plot_A_v_B_and_C_plotly_nb('Bz','Z','Y',False,0,'X==0','-800<Y<801','4400<Z<13500')

This is the heart of the DS.  A steep gradient field in the target region, and a constant field in the tracker region.

In [ ]:
plot_maker.plot_A_v_B_and_C_plotly_nb('Br','Z','Y',False,0,'X==0','-800<Y<801','4400<Z<13500')

In [ ]:
plot_maker.plot_A_v_B_and_C_plotly_nb('Bphi','Z','Y',False,0,'X==0','-800<Y<801','4400<Z<13500')

Obviously, the DS is not an ideal solenoid, and has non-negligible contributions to its radial and azimuthal magnetic field components.  However, there are no currents or magnetic sources located within the region of interest, so Laplace's equation is still valid.  This means that the 3D, 3-component magnetic field vector should be derivable from the partial derivatives of the scalar field solution to Laplace's equation in this region.

## Formulating a Semi-analytical Expression

Laplace's equation is simply:
$$\nabla^2\Phi = 0$$
Where $\Phi$ is the scalar magnetic field potential, which implies that:
$$ B_z = \frac{\partial}{\partial z}\Phi,\quad B_r = \frac{\partial}{\partial r}\Phi,\quad B_\phi = \frac{\partial}{\partial \phi}\Phi$$

The solutions to Laplace's equation are separable in a host of different coordinate systems.  Given the nature of our particular problem, cylindrical coordinates seem like the natural choice, and yield a nice series solution:

$$ \Phi = \sum_{n=0}^\infty \sum_{m=1}^\infty \big(\sin(n\phi+\delta_n)\big)\big(E_{nm}I_n(k_{m}r)+F_{nm}K_n(k_{m}r)\big)\big(A_{nm}\sin(k_{m}z)+B_{nm}\cos(k_{m}z)\big)$$
where $I_n$ and $K_n$ are the modified bessel functions of the first and second kind, respectively, and $A-F$ are free parameters.

The $K_n$ terms can immediately be excluded, because they all diverge as $r\rightarrow0$.  This means that $F_{nm} =0$ and $E_{nm}$ can be reabsorbed into the $A,B$ parameters.  In order to guarantee that the scalar field satisfies some constant boundary conditions at large Z distances, we define $k_m$ as:
$$ k_m=\frac{m\pi}{L_{eff}} $$
where $L_{eff}$ is an effective length that is determined empirically.

There is also a valid generic solution for Laplace's equation in cylindrical coordinates that is expressed in terms of regular bessel functions and hyperbolic trig functions.  But given the oscillatory nature of the field distributions as a function of Z, it seems more natural to use a solution that invokes normal trig functions.

## Simulating the Field Mapper Data

A mock-up of the FMS measurement device for the DS is shown below:

![FMS measurement device](fms_mockup.png)

Five 3D hall probes are located along the long green arm in the above picture.  They are able to measure the $B_z, B_r, B_\phi$ components at each probe location.  The probes are located at fixed radii, ranging from ~25mm to ~800mm.  The whole apparatus travels in the Z direction at a predetermined step size, and the arms rotate in the $\phi$ direction to cover the entire $2\pi$ range.  At every step in Z, the arms rotate through some number of $\phi$ positions, and the hall probes record the magnetic field values at each step.  The end result yields a sparse 3D map of the 3 field components.  We can perform a regular sampling from the magnetic field simulation above to simulate the results of the field mapper: